# 🎨 Day 3: Fashion AI Training - Google Colab 版本

## 📋 概述
這個 Notebook 提供了在 Google Colab 上進行 Stable Diffusion v1.5 時尚微調的完整流程。

### ✨ 特色功能:
- 🔧 自動偵測並優化 GPU 配置 (T4/V100/A100)
- 💾 LoRA 高效微調 (節省記憶體)
- 🎯 FashionCLIP 智能特徵提取
- 📊 實時訓練監控
- 💽 自動 Google Drive 同步
- 📦 一鍵下載訓練結果

### 🔧 系統需求:
- Google Colab Pro (推薦，但免費版也可用)
- GPU 運行時 (T4/V100/A100)
- 約 2-5GB Google Drive 空間

## 🚀 步驟 1: 環境設置和檢查

In [ ]:
# 安裝必要套件
!pip install -q diffusers[torch]==0.21.4 transformers==4.35.2
!pip install -q accelerate==0.24.1 peft==0.6.2
!pip install -q xformers --index-url https://download.pytorch.org/whl/cu118
!pip install -q matplotlib seaborn

print("✅ 套件安裝完成")

In [ ]:
import torch
import os

# 檢查 GPU 狀態
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🔧 GPU: {gpu_name}")
    print(f"💾 VRAM: {gpu_memory:.1f} GB")
    
    # 自動配置建議
    if "T4" in gpu_name:
        print("🎯 建議使用 LoRA rank=4, batch_size=1")
    elif "V100" in gpu_name:
        print("🎯 建議使用 LoRA rank=8, batch_size=2")
    elif "A100" in gpu_name:
        print("🎯 建議使用 LoRA rank=16, batch_size=4")
else:
    print("❌ 沒有可用的 GPU，請確認運行時設置")

In [ ]:
# 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 創建工作目錄
work_dir = "/content/drive/MyDrive/fashion_ai_training"
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"📁 工作目錄: {work_dir}")
print(f"📍 當前位置: {os.getcwd()}")

## 📥 步驟 2: 下載主要訓練腳本

In [ ]:
# 創建主要訓練腳本 (直接在 Colab 中定義)
script_content = '''
# 這裡會包含完整的 day3_colab_finetuning.py 內容
# (由於內容過長，實際使用時建議直接運行下面的 wget 命令下載)
'''

# 或者從 GitHub 下載 (如果您將代碼上傳到 GitHub)
# !wget -O day3_colab_finetuning.py "您的GitHub原始文件連結"

# 或者直接貼上完整代碼
with open('day3_colab_finetuning.py', 'w', encoding='utf-8') as f:
    f.write(open('/content/day3_colab_finetuning.py').read())

print("✅ 主要腳本已準備")

## 📤 步驟 3: 上傳訓練圖片

請上傳您要用於訓練的時尚圖片 (建議 10-50 張)

In [ ]:
from google.colab import files
import shutil

# 上傳圖片檔案
print("📤 請選擇並上傳訓練圖片...")
uploaded = files.upload()

# 整理上傳的圖片
upload_dir = "uploaded_images"
os.makedirs(upload_dir, exist_ok=True)

image_files = []
for filename, content in uploaded.items():
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        file_path = os.path.join(upload_dir, filename)
        with open(file_path, 'wb') as f:
            f.write(content)
        image_files.append(file_path)
        print(f"✅ {filename}")

print(f"\n📊 總共上傳 {len(image_files)} 張圖片")

if len(image_files) == 0:
    print("❌ 沒有有效的圖片檔案，請重新上傳")
else:
    print("✅ 圖片上傳完成，可以開始訓練")

## ⚡ 步驟 4: 快速開始訓練 (一鍵執行)

In [ ]:
# 快速訓練模式 - 自動化執行所有步驟
import sys
sys.path.append('/content')

from day3_colab_finetuning import FashionSDFineTuner

# 初始化訓練器 (自動優化配置)
trainer = FashionSDFineTuner()

# 使用上傳的圖片
if 'image_files' in locals() and image_files:
    print(f"🖼️ 使用 {len(image_files)} 張上傳的圖片")
    
    # 提取特徵生成描述
    print("\n🔍 正在提取圖片特徵...")
    captions = trainer.extract_features_from_images(image_files)
    
    # 準備數據集
    print("\n📊 準備訓練數據...")
    dataloader = trainer.prepare_dataset(image_files, captions)
    
    # 開始訓練
    print("\n🚀 開始微調訓練...")
    print("⏰ 預計訓練時間: 30-60 分鐘 (取決於 GPU 和圖片數量)")
    
    trainer.train(dataloader)
    
    print("\n🎉 訓練完成！")
    
else:
    print("❌ 請先上傳訓練圖片")

## 🔧 步驟 5: 進階配置 (可選)

如果您想自定義訓練參數，可以使用這個部分

In [ ]:
# 自定義訓練配置
custom_config = {
    "num_epochs": 30,           # 訓練輪數
    "learning_rate": 1e-4,      # 學習率
    "lora_rank": 8,             # LoRA rank (越高越詳細但需更多記憶體)
    "train_batch_size": 1,      # 批次大小
    "gradient_accumulation_steps": 4,  # 梯度累積
    "save_steps": 50,           # 保存頻率
    "validation_steps": 25      # 驗證頻率
}

print("⚙️ 自定義配置:")
for key, value in custom_config.items():
    print(f"   {key}: {value}")

# 使用自定義配置初始化訓練器
# trainer_custom = FashionSDFineTuner(config=custom_config)
print("\n💡 如需使用自定義配置，請取消註釋上面的代碼行")

## 📊 步驟 6: 訓練監控

檢查訓練進度和結果

In [ ]:
# 檢查訓練結果
import matplotlib.pyplot as plt
from PIL import Image
import glob

# 顯示訓練進度圖表
if os.path.exists("models/training_progress.png"):
    img = Image.open("models/training_progress.png")
    plt.figure(figsize=(12, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title("Training Progress")
    plt.show()
else:
    print("📊 訓練圖表尚未生成")

# 顯示驗證圖片
validation_images = glob.glob("models/validation/*.png")
if validation_images:
    print(f"🖼️ 找到 {len(validation_images)} 張驗證圖片")
    
    # 顯示最新的幾張
    latest_images = sorted(validation_images)[-6:]  # 最新 6 張
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, img_path in enumerate(latest_images):
        if i < 6:
            img = Image.open(img_path)
            axes[i].imshow(img)
            axes[i].set_title(os.path.basename(img_path))
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("🖼️ 驗證圖片尚未生成")

## 🎨 步驟 7: 測試訓練好的模型

使用訓練好的模型生成新的時尚圖片

In [ ]:
from diffusers import StableDiffusionPipeline, DDPMScheduler
from peft import PeftModel

# 載入訓練好的模型
def load_trained_model():
    try:
        # 載入基礎模型
        pipeline = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None,
            requires_safety_checker=False
        )
        
        # 載入 LoRA 權重
        if os.path.exists("models/final_model"):
            pipeline.unet = PeftModel.from_pretrained(
                pipeline.unet, 
                "models/final_model"
            )
            print("✅ LoRA 權重載入成功")
        else:
            print("⚠️ 找不到訓練好的模型，使用原始模型")
        
        pipeline = pipeline.to("cuda")
        return pipeline
        
    except Exception as e:
        print(f"❌ 模型載入失敗: {e}")
        return None

# 載入模型
pipeline = load_trained_model()

if pipeline:
    # 測試提示詞
    test_prompts = [
        "a woman wearing an elegant black dress",
        "a man in casual blue shirt and jeans",
        "person in formal business suit",
        "stylish outfit with modern fashion"
    ]
    
    print("🎨 生成測試圖片...")
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, prompt in enumerate(test_prompts):
        print(f"   生成: {prompt}")
        
        image = pipeline(
            prompt,
            num_inference_steps=20,
            guidance_scale=7.5,
            width=512,
            height=512
        ).images[0]
        
        axes[i].imshow(image)
        axes[i].set_title(prompt[:30] + "..." if len(prompt) > 30 else prompt)
        axes[i].axis('off')
        
        # 保存圖片
        os.makedirs("generated_tests", exist_ok=True)
        image.save(f"generated_tests/test_{i+1}.png")
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 測試圖片生成完成")
else:
    print("❌ 無法載入模型進行測試")

## 📦 步驟 8: 下載訓練結果

打包並下載所有訓練結果

In [ ]:
import zipfile
from datetime import datetime

# 創建下載包
def create_download_package():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    package_name = f"fashion_ai_model_{timestamp}.zip"
    
    print(f"📦 創建下載包: {package_name}")
    
    with zipfile.ZipFile(package_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # 模型檔案
        if os.path.exists("models/final_model"):
            for root, dirs, files in os.walk("models/final_model"):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, "models")
                    zipf.write(file_path, f"model/{arcname}")
            print("   ✅ 模型檔案已添加")
        
        # 訓練圖表
        if os.path.exists("models/training_progress.png"):
            zipf.write("models/training_progress.png", "training_progress.png")
            print("   ✅ 訓練圖表已添加")
        
        # 驗證圖片
        validation_dir = "models/validation"
        if os.path.exists(validation_dir):
            for file in os.listdir(validation_dir):
                file_path = os.path.join(validation_dir, file)
                zipf.write(file_path, f"validation/{file}")
            print("   ✅ 驗證圖片已添加")
        
        # 測試生成圖片
        if os.path.exists("generated_tests"):
            for file in os.listdir("generated_tests"):
                file_path = os.path.join("generated_tests", file)
                zipf.write(file_path, f"test_generations/{file}")
            print("   ✅ 測試圖片已添加")
        
        # 使用說明
        readme_content = f"""
# Fashion AI 訓練結果

訓練時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
GPU: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}

## 檔案說明:
- model/ : LoRA 權重檔案
- validation/ : 訓練過程中的驗證圖片
- test_generations/ : 最終測試生成的圖片
- training_progress.png : 訓練損失曲線

## 使用方法:
1. 安裝 diffusers, transformers, peft
2. 載入基礎 SD v1.5 模型
3. 使用 PeftModel.from_pretrained() 載入 LoRA 權重
4. 即可開始生成時尚圖片
"""
        
        zipf.writestr("README.md", readme_content)
        print("   ✅ 說明文件已添加")
    
    return package_name

# 創建並下載
package_name = create_download_package()
print(f"\n📦 打包完成: {package_name}")

# 下載檔案
files.download(package_name)
print("✅ 下載開始")

## 🧹 步驟 9: 清理 (可選)

清理暫存檔案以節省 Google Drive 空間

In [ ]:
import shutil

# 詢問是否清理
cleanup = input("是否清理暫存檔案以節省空間? (y/N): ").lower().strip()

if cleanup == 'y' or cleanup == 'yes':
    print("🧹 清理中...")
    
    # 清理項目
    cleanup_items = [
        "uploaded_images",  # 原始上傳圖片
        "/content/cache",   # 模型快取
        "models/checkpoint-*",  # 中間檢查點
    ]
    
    for item in cleanup_items:
        if os.path.exists(item):
            if os.path.isdir(item):
                shutil.rmtree(item)
                print(f"   🗑️ 已刪除目錄: {item}")
            else:
                os.remove(item)
                print(f"   🗑️ 已刪除檔案: {item}")
    
    # 清理 GPU 記憶體
    if 'pipeline' in locals():
        del pipeline
    if 'trainer' in locals():
        del trainer
    
    import gc
    gc.collect()
    torch.cuda.empty_cache()
    
    print("✅ 清理完成")
else:
    print("🗂️ 保留所有檔案")

## 🎉 總結

恭喜！您已經成功完成了 Stable Diffusion v1.5 的時尚微調訓練。

### ✅ 完成的工作:
1. 🔧 自動配置 Colab 環境
2. 📤 上傳並處理訓練圖片
3. 🎯 使用 FashionCLIP 提取特徵
4. 🚀 執行 LoRA 微調訓練
5. 📊 監控訓練進度
6. 🎨 測試生成效果
7. 📦 下載完整結果

### 💡 後續使用:
- 下載的模型可以在本地使用
- 支援各種 Stable Diffusion 界面
- 可以進一步微調或與其他 LoRA 合併

### 🆘 如果遇到問題:
1. 檢查 GPU 記憶體是否足夠
2. 確認所有套件版本相容
3. 重新啟動運行時並重新執行
4. 調整批次大小或 LoRA rank

感謝使用 Fashion AI Training Suite! 🎨✨